# Tony Holdroyd

### Coursera IBM DS Capstone Week 3

### Week 3 Part 1

### I copied the table directly from the HTML into the file 'toronto.html'

In [1]:
import pandas as pd

### read in table, appears as a list

In [2]:
file = 'toronto.html'
df = pd.read_html(file)
df = df[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### drop 'Not assigned' Borough rows

In [3]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)

### assign not assigned Neighbourhoods to respective Boroughs

In [4]:
df['Neighbourhood'] = df.apply(lambda x : x['Borough'] if x['Neighbourhood']=='Not assigned' else x['Neighbourhood'], axis=1)


In [5]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### collect Neighbourhoods into lists per Postcode

In [6]:
df = df.groupby(['Postcode','Borough' ])['Neighbourhood'].apply(', '.join).reset_index()

In [7]:
df.head(25)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
df.shape # 103 rows by 3 cols seems to be the required answer?

(103, 3)

### Week 3 Part 2

### get lat and long

In [9]:
lat_long = pd.read_csv('Geospatial_Coordinates.csv')
lat_long.drop('Postal Code', axis=1, inplace=True)

### join tables

In [10]:
df1=pd.concat([df, lat_long], axis=1)
 

In [11]:
df1

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Week 3 Part 3

In [12]:
from geopy.geocoders import Nominatim
import folium
import requests

In [13]:
neighbourhoods =df1.drop('Postcode', axis =1)

eb_data = neighbourhoods[neighbourhoods['Borough'] == 'Etobicoke'].reset_index(drop=True)
eb_data


,Borough,Neighbourhood,Latitude,Longitude
0,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
1,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
4,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999
5,Etobicoke,Islington Avenue,43.667856,-79.532242
6,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
7,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
8,Etobicoke,Westmount,43.696319,-79.532242
9,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724


In [14]:
#s = eb_data['Neighbourhood'].str.split(',').apply(pd.Series, 1).stack()
#s.index = s.index.droplevel(-1)
#s.name = 'Neighbourhood'
#del eb_data['Neighbourhood']
#eb_data = eb_data.join(s)


In [15]:
eb_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
1,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
4,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999


In [16]:
eb_data = eb_data[['Neighbourhood', 'Latitude', 'Longitude']].reset_index(drop=True)

In [17]:
eb_data.head()

,Neighbourhood,Latitude,Longitude
0,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
1,"Alderwood, Long Branch",43.602414,-79.543484
2,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
4,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999


In [18]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="sc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Etobicoke, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Etobicoke, Toronto are 43.67145915, -79.5524920661167.


In [19]:
# create map of Etobicoke using latitude and longitude values
map_eb= folium.Map(location=[latitude, longitude], zoom_start=10)

In [20]:
# add markers to map
for lat, lng, label in zip(eb_data['Latitude'], eb_data['Longitude'], eb_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_eb)  
    
map_eb

In [21]:
CLIENT_ID = 'AGTECTUHU2A21IV2GQJM4ON0HL0CYTXMHD3JDWRAJD3CQG04' #  Foursquare ID
CLIENT_SECRET = 'XUQMS5VZKXP4TYDKKLFFGASNRWSFAO4ZPYIE2KFP4VMVDBDI' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AGTECTUHU2A21IV2GQJM4ON0HL0CYTXMHD3JDWRAJD3CQG04
CLIENT_SECRET:XUQMS5VZKXP4TYDKKLFFGASNRWSFAO4ZPYIE2KFP4VMVDBDI


In [22]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
eb_venues = getNearbyVenues(names=eb_data['Neighbourhood'],
                                   latitudes=eb_data['Latitude'],
                                   longitudes=eb_data['Longitude'])

Humber Bay Shores, Mimico South, New Toronto
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor
Islington Avenue
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Westmount
Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown
Northwest


In [24]:
print(eb_venues.shape)
eb_venues.head()

(74, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Domino's Pizza,43.601676,-79.500908,Pizza Place
2,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
3,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Maple Leaf House,43.602040,-79.498678,American Restaurant


In [25]:
eb_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",10,10,10,10,10,10
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",8,8,8,8,8,8
"Humber Bay Shores, Mimico South, New Toronto",14,14,14,14,14,14
"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",2,2,2,2,2,2
"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",3,3,3,3,3,3
"Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor",13,13,13,13,13,13
Northwest,3,3,3,3,3,3
"The Kingsway, Montgomery Road, Old Mill North",4,4,4,4,4,4


In [26]:
print('There are {} uniques categories.'.format(len(eb_venues['Venue Category'].unique())))

There are 42 uniques categories.


## Analyze Each Neighborhood

In [27]:
# one hot encoding
eb_onehot = pd.get_dummies(eb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eb_onehot['Neighbourhood'] = eb_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [eb_onehot.columns[-1]] + list(eb_onehot.columns[:-1])
eb_onehot = eb_onehot[fixed_columns]

eb_onehot

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Café,...,River,Sandwich Place,Seafood Restaurant,Shopping Plaza,Skating Rink,Smoke Shop,Social Club,Supplement Shop,Video Store,Wings Joint
0,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Humber Bay Shores, Mimico South, New Toronto",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Humber Bay Shores, Mimico South, New Toronto",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,"Humber Bay Shores, Mimico South, New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
eb_grouped = eb_onehot.groupby('Neighbourhood').mean().reset_index()
eb_grouped

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,Café,...,River,Sandwich Place,Seafood Restaurant,Shopping Plaza,Skating Rink,Smoke Shop,Social Club,Supplement Shop,Video Store,Wings Joint
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,...,0.00,0.111111,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.111111,0.000000
1,"Alderwood, Long Branch",0.000000,0.1,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00,0.100000,0.000000,0.000,0.1,0.00,0.000000,0.000000,0.000000,0.000000
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.000000,0.0,0.000000,0.000000,0.0,0.125000,0.000000,0.000000,0.125000,...,0.00,0.000000,0.000000,0.125,0.0,0.00,0.000000,0.000000,0.000000,0.000000
3,"Humber Bay Shores, Mimico South, New Toronto",0.071429,0.0,0.071429,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,...,0.00,0.071429,0.071429,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000
4,"Humber Bay, King's Mill Park, Kingsway Park So...",0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000
5,"Kingsview Village, Martin Grove Gardens, Richv...",0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000
6,"Kingsway Park South West, Mimico NW, The Queen...",0.000000,0.0,0.076923,0.000000,0.0,0.000000,0.076923,0.076923,0.000000,...,0.00,0.076923,0.000000,0.000,0.0,0.00,0.076923,0.076923,0.000000,0.076923
7,Northwest,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000
8,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.25,0.000000,0.000000,0.000,0.0,0.25,0.000000,0.000000,0.000000,0.000000
9,Westmount,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00,0.125000,0.000000,0.000,0.0,0.00,0.000000,0.000000,0.000000,0.000000


In [29]:
eb_grouped.shape

(10, 43)

In [30]:
num_top_venues = 5

for hood in eb_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = eb_grouped[eb_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
           venue  freq
0  Grocery Store  0.22
1    Video Store  0.11
2       Pharmacy  0.11
3     Beer Store  0.11
4    Pizza Place  0.11


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place   0.2
1  Athletics & Sports   0.1
2                 Gym   0.1
3            Pharmacy   0.1
4        Skating Rink   0.1


----Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe----
          venue  freq
0  Liquor Store  0.12
1      Pharmacy  0.12
2   Pizza Place  0.12
3    Beer Store  0.12
4          Café  0.12


----Humber Bay Shores, Mimico South, New Toronto----
                  venue  freq
0                  Café  0.14
1   American Restaurant  0.07
2  Fast Food Restaurant  0.07
3    Seafood Restaurant  0.07
4        Sandwich Place  0.07


----Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway E

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10
import numpy as np
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = eb_grouped['Neighbourhood']

for ind in np.arange(eb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Video Store,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Sandwich Place,Pharmacy,Pizza Place,Construction & Landscaping,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place,Coffee Shop,Pharmacy,Pool,Gym,Skating Rink,Athletics & Sports,Beer Store
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Pizza Place,Liquor Store,Shopping Plaza,Beer Store,Convenience Store,Café,Pharmacy,Coffee Shop,Wings Joint,Fried Chicken Joint
3,"Humber Bay Shores, Mimico South, New Toronto",Café,Restaurant,Gym,Liquor Store,Pet Store,Pharmacy,Pizza Place,Fried Chicken Joint,American Restaurant,Sandwich Place
4,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Construction & Landscaping,Wings Joint,Coffee Shop,Gym,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store


## 4. Cluster Neighborhoods

In [33]:
# set number of clusters
# import k-means from clustering stage
from sklearn.cluster import KMeans
kclusters = 5

eb_grouped_clustering = eb_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 3, 0, 2, 4, 1, 2], dtype=int32)

In [34]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Video Store,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Sandwich Place,Pharmacy,Pizza Place,Construction & Landscaping,Drugstore
1,2,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place,Coffee Shop,Pharmacy,Pool,Gym,Skating Rink,Athletics & Sports,Beer Store
2,2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Pizza Place,Liquor Store,Shopping Plaza,Beer Store,Convenience Store,Café,Pharmacy,Coffee Shop,Wings Joint,Fried Chicken Joint
3,2,"Humber Bay Shores, Mimico South, New Toronto",Café,Restaurant,Gym,Liquor Store,Pet Store,Pharmacy,Pizza Place,Fried Chicken Joint,American Restaurant,Sandwich Place
4,3,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Construction & Landscaping,Wings Joint,Coffee Shop,Gym,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store
5,0,"Kingsview Village, Martin Grove Gardens, Richv...",Mobile Phone Shop,Park,Pizza Place,Wings Joint,Gym,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store
6,2,"Kingsway Park South West, Mimico NW, The Queen...",Wings Joint,Sandwich Place,Grocery Store,Fast Food Restaurant,Discount Store,Convenience Store,Gym,Burger Joint,Burrito Place,Bakery
7,4,Northwest,Rental Car Location,Bar,Drugstore,Wings Joint,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Convenience Store,Construction & Landscaping
8,1,"The Kingsway, Montgomery Road, Old Mill North",Pool,Smoke Shop,Park,River,Wings Joint,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store
9,2,Westmount,Pizza Place,Intersection,Sandwich Place,Middle Eastern Restaurant,Discount Store,Coffee Shop,Chinese Restaurant,Burger Joint,Beer Store,Burrito Place


In [35]:

eb_merged = eb_data

In [36]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
eb_merged.set_index('Neighbourhood')
neighborhoods_venues_sorted.set_index('Neighbourhood')
eb_merged = pd.merge(eb_merged,neighborhoods_venues_sorted, on='Neighbourhood')
eb_merged.head()


,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,2,Café,Restaurant,Gym,Liquor Store,Pet Store,Pharmacy,Pizza Place,Fried Chicken Joint,American Restaurant,Sandwich Place
1,"Alderwood, Long Branch",43.602414,-79.543484,2,Pizza Place,Pub,Sandwich Place,Coffee Shop,Pharmacy,Pool,Gym,Skating Rink,Athletics & Sports,Beer Store
2,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1,Pool,Smoke Shop,Park,River,Wings Joint,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store
3,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,3,Baseball Field,Construction & Landscaping,Wings Joint,Coffee Shop,Gym,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store
4,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999,2,Wings Joint,Sandwich Place,Grocery Store,Fast Food Restaurant,Discount Store,Convenience Store,Gym,Burger Joint,Burrito Place,Bakery


In [37]:


eb_merged.head() # check the last columns!
#eb_merged['Cluster Labels'] = int(eb_merged['Cluster Labels'] )

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,2,Café,Restaurant,Gym,Liquor Store,Pet Store,Pharmacy,Pizza Place,Fried Chicken Joint,American Restaurant,Sandwich Place
1,"Alderwood, Long Branch",43.602414,-79.543484,2,Pizza Place,Pub,Sandwich Place,Coffee Shop,Pharmacy,Pool,Gym,Skating Rink,Athletics & Sports,Beer Store
2,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1,Pool,Smoke Shop,Park,River,Wings Joint,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store
3,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,3,Baseball Field,Construction & Landscaping,Wings Joint,Coffee Shop,Gym,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store
4,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999,2,Wings Joint,Sandwich Place,Grocery Store,Fast Food Restaurant,Discount Store,Convenience Store,Gym,Burger Joint,Burrito Place,Bakery


In [38]:
# create map

import matplotlib.cm as cm
import matplotlib.colors as colors
import math
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eb_merged['Latitude'], eb_merged['Longitude'], eb_merged['Neighbourhood'], eb_merged['Cluster Labels']):
        print(lat, lon, poi, cluster)
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

43.6056466 -79.50132070000001 Humber Bay Shores, Mimico South, New Toronto 2
43.60241370000001 -79.54348409999999 Alderwood, Long Branch 2
43.653653600000005 -79.5069436 The Kingsway, Montgomery Road, Old Mill North 1
43.6362579 -79.49850909999999 Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea 3
43.6288408 -79.52099940000001 Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor 2
43.6435152 -79.57720079999999 Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe 2
43.696319 -79.53224240000002 Westmount 2
43.6889054 -79.55472440000001 Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips 0
43.739416399999996 -79.5884369 Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown 2
43.706748299999994 -79.5940544 Northwest 4


### Cluster 1

In [39]:
eb_merged.loc[eb_merged['Cluster Labels'] == 0, eb_merged.columns[[0] + list(range(4, eb_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"Kingsview Village, Martin Grove Gardens, Richv...",Mobile Phone Shop,Park,Pizza Place,Wings Joint,Gym,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store


### Cluster 2

In [40]:
eb_merged.loc[eb_merged['Cluster Labels'] == 1, eb_merged.columns[[0] + list(range(4, eb_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"The Kingsway, Montgomery Road, Old Mill North",Pool,Smoke Shop,Park,River,Wings Joint,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store


### Cluster 3

In [41]:
eb_merged.loc[eb_merged['Cluster Labels'] == 2, eb_merged.columns[[0] + list(range(4, eb_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Humber Bay Shores, Mimico South, New Toronto",Café,Restaurant,Gym,Liquor Store,Pet Store,Pharmacy,Pizza Place,Fried Chicken Joint,American Restaurant,Sandwich Place
1,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place,Coffee Shop,Pharmacy,Pool,Gym,Skating Rink,Athletics & Sports,Beer Store
4,"Kingsway Park South West, Mimico NW, The Queen...",Wings Joint,Sandwich Place,Grocery Store,Fast Food Restaurant,Discount Store,Convenience Store,Gym,Burger Joint,Burrito Place,Bakery
5,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Pizza Place,Liquor Store,Shopping Plaza,Beer Store,Convenience Store,Café,Pharmacy,Coffee Shop,Wings Joint,Fried Chicken Joint
6,Westmount,Pizza Place,Intersection,Sandwich Place,Middle Eastern Restaurant,Discount Store,Coffee Shop,Chinese Restaurant,Burger Joint,Beer Store,Burrito Place
8,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Video Store,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Sandwich Place,Pharmacy,Pizza Place,Construction & Landscaping,Drugstore


### Cluster 4

In [42]:
eb_merged.loc[eb_merged['Cluster Labels'] == 3, eb_merged.columns[[0] + list(range(4, eb_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Construction & Landscaping,Wings Joint,Coffee Shop,Gym,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Drugstore,Discount Store


### Cluster 5

In [43]:
eb_merged.loc[eb_merged['Cluster Labels'] == 4, eb_merged.columns[[0] + list(range(4, eb_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Northwest,Rental Car Location,Bar,Drugstore,Wings Joint,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Discount Store,Convenience Store,Construction & Landscaping


In [ ]:
## Notes:
#